<a href="https://colab.research.google.com/github/ejrtks1020/Pytorch-basic/blob/main/PyTorch_CNN_MNIST_2_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. 모델 이해하기

* 모델의 아키텍처는 총 5개의 층으로 구성 된다
* 챕터 17에서 배운 모델의 1번 레이어와 2번레이어는 동일하되, 새로운 합성곱층과 전결합층을 추가한다.

##1번 레이어 : 합성곱층 (Convolutional layer)

합성곱(in_channel = 1, out_channel = 32, kernel_size = 3, stride = 1, padding = 1) + 활성화 함수 ReLU

맥스풀링(kernel_size = 2, stride = 2)

##2번 레이어 : 합성곱층 (Convolutional layer)

합성곱(in_channel = 32, out_channel = 64, kernel_size = 3, stride = 1, padding = 1) + 활성화 함수 ReLU

맥스풀링(kernel_size = 2, stride = 2)

##3번 레이어 : 합성곱층 (Convolutional layer)

합성곱(in_channel = 64, out_channel = 128, kernel_size = 3, stride = 1, padding = 1) + 활성화 함수 ReLU

맥스풀링(kernel_size = 2, stride = 2)

##4번 레이어 : 전결합층 (Fully-Connected Layer)
특성맵을 펼친다. - batch_size x 4 x 4 x 128 -> batch_size x 2048

전결합층(뉴런 625개) + 활성화 함수 ReLU

##5번 레이어 : 전결합층 (Fully-Connected Layer)
전결합층(뉴런 10개) + 활성화 함수 softmax




# 2. 깊은 CNN으로 MNIST 분류하기

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

#랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [ ]:
#하이퍼 파라미터 설정
learning_rate = 0.001
training_epoch = 15
batch_size = 100

In [ ]:
mnist_train = dsets.MNIST(root = 'MNIST_data/',         #다운로드 경로지정
                          train=True,
                          transform = transforms.ToTensor(), #텐서로 변환
                          download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',    
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [ ]:
data_loader = torch.utils.data.DataLoader(mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [ ]:
#클래스로 모델 설계

class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.keep_prob = 0.5
    # L1 ImgIn shape = (? , 28, 28, 1)
    # Conv           = (? , 28, 28, 32)
    # Pool           = (? , 14, 14, 32)
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
    )
    # L2 ImgIn shape = (? , 14, 14, 32)
    # Conv           = (? , 14, 14, 64)
    # Pool           = (? , 7, 7, 64)
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
    )

    # L3 ImgIn shape = (? , 7, 7, 64)
    # Conv           = (? , 7, 7, 128)
    # Pool           = (? , 4, 4, 128)
    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1)
    )

    # L4 FC 4x4x128 inputs -> 625 outputs
    self.fc1 = torch.nn.Linear(4 * 4 *128, 625, bias = True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    self.layer4 = torch.nn.Sequential(
        self.fc1,
        torch.nn.ReLU(),
        torch.nn.Dropout(p = 1 - self.keep_prob)
    )

    # L5 Final FC 625 inputs -> 10 outputs
    self.fc2 = torch.nn.Linear(625, 10, bias = True)
    torch.nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0), -1) # Flatten them for FC
    out = self.layer4(out)
    out = self.fc2(out)

    return out


In [ ]:
# 모델을 정의
model = CNN().to(device)

In [ ]:
# 비용함수와 옵티마이저 정의
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
# 총 배치의 수 출력
total_batch = len(data_loader)
print(total_batch)

600


In [ ]:
for epoch in range(training_epoch):
  avg_cost = 0

  for X, y in data_loader:
    X = X.to(device)
    y = y.to(device)

    hypothesis = model(X)

    optimizer.zero_grad()
    cost = criterion(hypothesis , y)
    cost.backward()
    optimizer.step()
    
    avg_cost += cost.item() / total_batch

  print('[Epoch : {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch :    1] cost = 0.19175734
[Epoch :    2] cost = 0.0535033985
[Epoch :    3] cost = 0.0368709617
[Epoch :    4] cost = 0.0303774569
[Epoch :    5] cost = 0.023499894
[Epoch :    6] cost = 0.0210392055
[Epoch :    7] cost = 0.0178072494
[Epoch :    8] cost = 0.0148930046
[Epoch :    9] cost = 0.0120501581
[Epoch :   10] cost = 0.0102065319
[Epoch :   11] cost = 0.0123608552
[Epoch :   12] cost = 0.00805815414
[Epoch :   13] cost = 0.00899475507
[Epoch :   14] cost = 0.00757733716
[Epoch :   15] cost = 0.008292071


In [ ]:
# 테스트
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test),1, 28, 28).float().to(device)
  y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == y_test
  print(correct_prediction)
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


tensor([True, True, True,  ..., True, True, True], device='cuda:0')
Accuracy: 0.9817999601364136


층을 더 깊게 쌓았는데 오히려 정확도가 소폭 감소 하였다. 층을 깊게 쌓는 것도 중요하지만 꼭 깊게 쌓는 것이 정확도를 올려주지는 않으며 효율저긍로 쌓는것도 중요하다.